### CARGAR DEPENDENCIAS

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
import torch.optim as optim
import numpy as np
from model import SimpleRNN, RacingDataset
import os
import torchvision.transforms.functional as F
from torchvision import transforms, models
import time
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import random

### DEFINICION DE PARAMETROS

In [5]:
#pretrained_cnn = models.efficientnet_v2_s(weights='IMAGENET1K_V1') 
#pretrained_cnn = models.efficientnet_b0(weights='IMAGENET1K_V1')

cnn_name = "efficientnet_b0" #"efficientnet_v2_s"
hidden_size = 256 # Número de neuronas en la capa oculta 512 usado por Iker
output_size = 2 # Giro y aceleración
input_size = (240, 135)  # 16:9 ratio
num_layers = 1
dropout = 0 # Se necesita num_layers > 1 para que el dropout tenga efecto
bias = True
cnn_train = True # Si se desea entrenar la parte CNN 

seq_len = 5 # Número de imágenes a considerar en la secuencia
batch_size = 32 # Número de secuencias a considerar en paralelo
num_epochs = 30 # Número de veces que se recorrerá el dataset
learning_rate = 0.001 

# Parámetros del early stopping
patience = 5  # Tolerancia de 5 epochs sin mejora
best_val_loss = float('inf')  # Inicia con el peor valor posible
early_stop_counter = 0  # Contador de epochs sin mejora

# Definir ponderaciones para las etiquetas (ejemplo para 9 etiquetas)
""" weights = [ 1.0, # -
            2.0, # A
            2.0, # D
            0.8, # W
            2.0, # S
            1.0, # WA
            1.5, # SA
            3, # WD
            1.0] # SD """


# Definir las rutas de los directorios de datos y de guardado de modelos
train_data_dir = "./datasets/train" #train_dataset"
validation_data_dir = "./datasets/validation"
save_dir = "./trained_models"

#model_name = f"Oval_Model_4_EffNet_b0_CNNGRAD_{hidden_size}_epoch"
model_name = f"Model-{cnn_name}-{seq_len}-{input_size[0]}-{input_size[1]}-{output_size}-{hidden_size}-epoch"
print(f"{model_name}")

os.makedirs(save_dir, exist_ok=True)

# Definir el escritor de TensorBoard para visualización
writer = SummaryWriter(log_dir="./runs/" + model_name) 

Model-efficientnet_b0-5-240-256-epoch


### INICIAR MODELO Y CARGAR DATOS

In [6]:
# Inicializar el modelo
model = SimpleRNN(cnn_name, hidden_size, output_size, (3, *input_size), num_layers, dropout, bias)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Usando CUDA" if torch.cuda.is_available() else "USANDO CPU")


# Cargar los datasets
train_dataset = RacingDataset(data_dir= train_data_dir, seq_len= seq_len, input_size=input_size, controller = True)
test_dataset = RacingDataset(data_dir= validation_data_dir, seq_len= seq_len, input_size=input_size, controller = True)

# Crear los dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Optimización y función de pérdida
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#criterion = CrossEntropyLoss(weight=torch.tensor(weights).to(device))

criterion = nn.MSELoss()


#criterion = nn.CrossEntropyLoss()

Usando CUDA


### ENTRENAMIENTO

In [7]:
print("Iniciando entrenamiento...")

start_time = time.time()    # Tiempo de inicio del entrenamiento

# Ciclo de entrenamiento
for epoch in range(num_epochs): 
    
    # Iniciar buffer de secuencia
    sequence_buffer_train = []
    sequence_buffer_validation = []

    epoch_start_time = time.time()  # Tiempo de inicio del epoch

    model.train() # Establecer el modo de entrenamiento
    running_loss = 0.0
    try:
        for i, (images, labels) in enumerate(train_loader):

            print(f"Trabajando en Epoch {epoch+1}. Progreso: {(i+1)/len(train_loader)*100:.2f}%", end='\r')

            #print(f"Dimensiones de las etiquetas en el batch {i+1}: {labels.size()}")

            input_sequence = images # (seq_len, batch_size, channels, height, width)
            input_sequence, labels = input_sequence.to(device), labels.to(device) # Mover los datos al dispositivo            
        
            optimizer.zero_grad()
            
            # Habilitar precisión mixta
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_sequence)
                loss = criterion(outputs, labels) # Solo se considera la última etiqueta de la secuencia
            
            # Backpropagation y optimización
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        # Registrar la pérdida en TensorBoard
        writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)

        # Guardar el modelo después de cada epoch
        # torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_{epoch+1}.pth'))

        print("\nValidando modelo...")

        # Validación en el conjunto de datos de prueba
        model.eval()  # Establecer el modo de evaluación
        test_loss = 0.0
        with torch.no_grad():
            for i, (images, labels) in enumerate(validation_loader):
                
                print(f"Validando en Epoch {epoch+1}. Progreso: {(i+1)/len(validation_loader)*100:.2f}%", end='\r')

                input_test_sequence = images # (seq_len, batch_size, channels, height, width)
                input_test_sequence, labels = input_test_sequence.to(device), labels.to(device) # Mover los datos al dispositivo

                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(input_test_sequence)
                    loss = criterion(outputs, labels)
                    test_loss += loss.item()

        val_loss = test_loss / len(validation_loader)
        writer.add_scalar('Loss/test', val_loss, epoch)

        # Comprobar si early stopping es necesario
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0  # Restablecer el contador
            torch.save(model.state_dict(), os.path.join(save_dir, f'{model_name}_{epoch+1}.pth'))  # Guardar el mejor modelo
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping triggered after {patience} epochs without improvement.")
                break

        end_time = time.time() - start_time  # Tiempo de finalización del epoch
        # Convertir end_time a formato hh:mm:ss
        end_time = time.strftime("%H:%M:%S", time.gmtime(end_time))
        epoch_time = time.time() - epoch_start_time  # Tiempo de finalización del epoch
        epoch_time = time.strftime("%H:%M:%S", time.gmtime(epoch_time))
        print(f'\nEpoch [{epoch+1}/{num_epochs}], '
            f'Train Loss: {running_loss/len(train_loader):.4f}, '
            f'Val Loss: {test_loss/len(validation_loader):.4f}, '
            f'Epoch Time: {epoch_time}, '
            f'Total Time: {end_time}'
            )
    except KeyboardInterrupt:
        print("\nEntrenamiento interrumpido.")
        break
print("Entrenamiento terminado.")

writer.close()  # Cerrar el escritor de TensorBoard

Iniciando entrenamiento...
Trabajando en Epoch 1. Progreso: 100.00%
Validando modelo...
Validando en Epoch 1. Progreso: 100.00%
Epoch [1/30], Train Loss: 0.0665, Val Loss: 0.0604, Epoch Time: 00:24:35, Total Time: 00:24:35
Trabajando en Epoch 2. Progreso: 100.00%
Validando modelo...
Validando en Epoch 2. Progreso: 100.00%
Epoch [2/30], Train Loss: 0.0316, Val Loss: 0.0630, Epoch Time: 00:15:28, Total Time: 00:40:04
Trabajando en Epoch 3. Progreso: 100.00%
Validando modelo...
Validando en Epoch 3. Progreso: 100.00%
Epoch [3/30], Train Loss: 0.0281, Val Loss: 0.0686, Epoch Time: 00:14:38, Total Time: 00:54:42
Trabajando en Epoch 4. Progreso: 100.00%
Validando modelo...
Validando en Epoch 4. Progreso: 100.00%
Epoch [4/30], Train Loss: 0.0228, Val Loss: 0.0542, Epoch Time: 00:14:53, Total Time: 01:09:35
Trabajando en Epoch 5. Progreso: 100.00%
Validando modelo...
Validando en Epoch 5. Progreso: 100.00%
Epoch [5/30], Train Loss: 0.0197, Val Loss: 0.0747, Epoch Time: 00:14:32, Total Time: 01